In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

stringify = lambda arr: "'" + "','".join(arr) + "'"

In [2]:
REPOS = [
    'https://github.com/0xproject/protocol',
    'https://github.com/aave-dao/aave-v3-origin',
    'https://github.com/across-protocol/contracts',
    'https://github.com/aerodrome-finance/contracts',
    'https://github.com/eth-infinitism/account-abstraction',
    'https://github.com/ionicprotocol/monorepo',
    'https://github.com/LayerZero-Labs/LayerZero-v2',
    'https://github.com/lifinance/contracts',
    'https://github.com/moonwell-fi/moonwell-contracts-v2',
    'https://github.com/moxie-protocol/contracts',
    'https://github.com/ourzora/zora-protocol',
    'https://github.com/projectopensea/seaport',
    'https://github.com/pyth-network/pyth-crosschain',
    'https://github.com/reservoirprotocol/marketplace-v2',
    'https://github.com/safe-global/safe-smart-account',
    'https://github.com/socketdottech/socket-dl',
    'https://github.com/stargate-protocol/stargate-v2',
    'https://github.com/sushi-labs/sushiswap',
    'https://github.com/thirdweb-dev/contracts',
    'https://github.com/uniswap/v4-core',
    'https://github.com/uniswap/v4-periphery',
    'https://github.com/virtual-protocol/protocol-contracts',
    'https://github.com/worldcoin/world-id-contracts'
]

In [3]:
repo_namespaces = set([x.split('/')[-2].lower() for x in REPOS])
repo_names = set([x.split('/')[-1].lower() for x in REPOS])

df_sbom = client.to_pandas(f"""
SELECT DISTINCT
  CONCAT(s.from_artifact_namespace, '/', s.from_artifact_name) AS dependent_repo,
  CONCAT(p.package_owner_artifact_namespace, '/', p.package_owner_artifact_name) AS dependency_repo,
  s.to_package_artifact_source AS package_source,
  s.to_package_artifact_name AS dependency_name
FROM sboms_v0 AS s
LEFT JOIN package_owners_v0 AS p
  ON p.package_artifact_source = s.to_package_artifact_source
  AND p.package_artifact_name = s.to_package_artifact_name
WHERE
  s.from_artifact_namespace IN ({stringify(repo_namespaces)})
  AND s.from_artifact_name IN ({stringify(repo_names)})
""")

df_sbom.tail()

,dependent_repo,dependency_repo,package_source,dependency_name
28304,across-protocol/contracts,sindresorhus/multimatch,NPM,multimatch
28305,socketdottech/socket-dl,moment/moment-timezone,NPM,moment-timezone
28306,pyth-network/pyth-crosschain,<NA>,CARGO,solana-runtime
28307,pyth-network/pyth-crosschain,<NA>,CARGO,cosmwasm-schema
28308,across-protocol/contracts,djc/rustc-version-rs,RUST,rustc_version


In [4]:
# https://docs.google.com/spreadsheets/d/1-iA72k57TGBxbxa54sx1kZKyWl-QHsDyAueja5x0VNo/edit?usp=sharing
df_openq = pd.read_csv('/Users/cerv1/Downloads/OpenQ Test Output 20 Repos - output.csv')
df_openq = df_openq[df_openq['url'].isin(REPOS)]
df_openq.reset_index(inplace=True, drop=True)
df_openq.tail()

,url,dependency_name,dependency_file
89,https://github.com/thirdweb-dev/contracts,solhint,package.json
90,https://github.com/thirdweb-dev/contracts,solidity-coverage,package.json
91,https://github.com/thirdweb-dev/contracts,@thirdweb-dev/chains,package.json
92,https://github.com/thirdweb-dev/contracts,@thirdweb-dev/merkletree,package.json
93,https://github.com/thirdweb-dev/contracts,@thirdweb-dev/sdk,package.json


In [5]:
merged = df_openq.merge(
    df_sbom[['dependency_name']],
    on='dependency_name',
    how='left',
    indicator=True
)

merged[merged['_merge'] == 'left_only']

,url,dependency_name,dependency_file,_merge
1,https://github.com/0xproject/protocol,@0x/contract-artifacts,package.json,left_only
74,https://github.com/0xproject/protocol,synthetix,package.json,left_only
361,https://github.com/ourzora/zora-protocol,@zoralabs/1155-deployments,package.json,left_only
362,https://github.com/ourzora/zora-protocol,@zoralabs/coins-sdk,package.json,left_only
363,https://github.com/ourzora/zora-protocol,@zoralabs/mints-contracts,package.json,left_only
364,https://github.com/ourzora/zora-protocol,@zoralabs/protocol-deployments,package.json,left_only
365,https://github.com/ourzora/zora-protocol,@zoralabs/protocol-rewards,package.json,left_only
366,https://github.com/ourzora/zora-protocol,@zoralabs/protocol-sdk,package.json,left_only
367,https://github.com/ourzora/zora-protocol,@zoralabs/zora-1155-contracts,package.json,left_only
478,https://github.com/pyth-network/pyth-crosschain,@pythnetwork/entropy-sdk-solidity,package.json,left_only
